In [41]:
import pandas as pd

# RECOVER DATASETS 

First step is to recover the datasets we have created and saved from the different queries and webscrapping.
A first view let us see several problems that must be solve before analysing the data:

    - Three variables must be standarized beteween all the dataframes for future comparison/analysis:
        - autonomous communities names
        - types of energy
        - units
    
    - Also in two of the dataframes data is not datetieme type so, it must be changed
    

In [42]:
spain_global = pd.read_csv('starting_data/spain_global.csv')
gen = pd.read_csv('data/no_clean/df_generated_monthly_ccaa.csv')
demand = pd.read_csv('data/no_clean/year_demand.csv')
global_web = pd.read_csv("starting_data/global/global_web.csv")
reservoir  = pd.read_excel('starting_data/Embalses.xlsx')

C:\Users\psaez\AppData\Local\Temp\ipykernel_35256\3424186575.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  global_web = pd.read_csv("starting_data/global/global_web.csv")


In [43]:
spain_global.head()

,project,type,country,unit,status,capacity,start_year,owner,lat,lng,url,aacc
0,Aboño power station,coal,Spain,Unit 1,operating,360.0,1974.0,HC Energia,43.55356,-5.72185,https://www.gem.wiki/Abo%C3%B1o_power_station,Asturias
1,Aboño power station,coal,Spain,Unit 2,operating,556.2,1985.0,HC Energia,43.55356,-5.72185,https://www.gem.wiki/Abo%C3%B1o_power_station,Asturias
2,Alcúdia power station,coal,Spain,Phase 2 Unit 1,retired,125.0,1981.0,GESA-Endesa,39.81075,3.09141,https://www.gem.wiki/Alcúdia_power_station,Baleares
3,Alcúdia power station,coal,Spain,Phase 2 Unit 2,retired,125.0,1982.0,GESA-Endesa,39.81075,3.09141,https://www.gem.wiki/Alcúdia_power_station,Baleares
4,Alcúdia power station,coal,Spain,Phase 2 Unit 3,operating,130.0,1997.0,GESA-Endesa,39.81075,3.09141,https://www.gem.wiki/Alcúdia_power_station,Baleares


In [44]:
gen.head()

,Unnamed: 0,class,type,date,value,percentage,aacc
0,0,Hidráulica,Renovable,2014-01-01,303.396,0.000421,Canarias
1,1,Motores diésel,No-Renovable,2014-01-01,181768.586,0.251957,Canarias
2,2,Turbina de gas,No-Renovable,2014-01-01,26245.404,0.036380,Canarias
3,3,Turbina de vapor,No-Renovable,2014-01-01,216697.507,0.300373,Canarias
4,4,Ciclo combinado,No-Renovable,2014-01-01,247173.544,0.342618,Canarias


In [45]:
demand.head()

,Unnamed: 0,class,type,date,value,percentage,aacc
0,0,Demanda,NaN,2014-01-01,721426.992,1,Canarias
1,1,Demanda,NaN,2014-01-01,424810.960,1,Baleares
2,2,Demanda,NaN,2014-01-01,18805.790,1,Ceuta
3,3,Demanda,NaN,2014-01-01,17999.596,1,Melilla
4,4,Demanda,NaN,2014-01-01,3406748.087,1,Andalucía


In [46]:
global_web.sample()

,project,type,country,unit,status,capacity,start_year,owner,lat,lng,url
25927,Bui Hydro Dam solar farm,solar,Ghana,MW,announced,60.0,NaN,Bui Power Authority (BPA),5.6221,-0.176,https://gem.wiki/Bui_Hydro_Dam_solar_farm


In [47]:
reservoir.sample()

,DEMARCACIÓN HIDROGRÁFICA TERRESTRE,Superficie total de los embalses de la Demaración Hidrográfica- km2,aacc
23,EL HIERRO,0.0,Canarias


## CLEANING DATA

#### 1- MATCH AUTONOMOUS COMMUNITIES

 We will do this in two steps: 
 
     1- use the same column name in all the dataframes
     2- make a dictionary to be apply if necessary to the dataframes to standarize autonomous communities names

In [51]:
spain_global.rename(columns={'ccaa':'aacc'}, inplace = True)

In [54]:
gen.rename(columns={"comunidad":'aacc'}, inplace = True)

In [60]:
aacc_dicc = {'Castilla_León': 'Castilla-Leon', 
             'Andalucía':'Andalucia',
             'Aragón':'Aragon',
            'País_Vasco':'Pais Vasco',
             'Castilla_la_Mancha':'Castilla-La Mancha', 
             'La_Rioja':'La Rioja',
             'Valenciana':'Valencia' }

In [61]:
spain_global['aacc'] = spain_global['aacc'].replace(aacc_dicc)

In [62]:
demand['aacc'] = demand['aacc'].replace(aacc_dicc)

In [63]:
gen['aacc'] = gen['aacc'].replace(aacc_dicc)

#### 2- 'date' columns to datetype

In [64]:
gen['date'] = pd.to_datetime(gen['date'])

In [65]:
demand['date'] = pd.to_datetime(demand['date'])

#### 3- type  of energy

Check the name of the types of energy in all dataframes:

In [68]:
global_web['type'].unique()

array(['coal', 'oil_gas', 'solar', 'wind', 'hydropower', 'geothermal',
       'bioenergy', 'nuclear'], dtype=object)

In [69]:
spain_global['type'].unique()

array(['coal', 'oil_gas', 'solar', 'wind', 'hydropower', 'bioenergy',
       'nuclear'], dtype=object)

In [70]:
gen['class'].unique()

array(['Hidráulica', 'Motores diésel', 'Turbina de gas',
       'Turbina de vapor', 'Ciclo combinado', 'Hidroeólica', 'Eólica',
       'Solar fotovoltaica', 'Otras renovables', 'Generación total',
       'Carbón', 'Cogeneración', 'Residuos no renovables',
       'Residuos renovables', 'Turbinación bombeo', 'Solar térmica',
       'Nuclear', 'Fuel + Gas'], dtype=object)

 we will try to unify as much as possible energies from gen dataframe and spain_global so , in order to do so we will:
 
     - change the name of the column 'type' in spain_global to class
     - consider both 'Solar Fotovoltaica' and 'Solar térmica' as 'solar'
  on the other hand we will create a dicctionary to replace names and unify whenever possible

In [71]:
spain_global.rename(columns ={'type':'class'}, inplace = True)

In [72]:
energy_dicc = {
    'Solar fotovoltaica':'solar',
    'Solar térmica': 'solar',
    'Hidroeólica':'hydropower',
    'Eolica': 'wind',
    'Carbón' : 'coal',
    'Nuclear' : 'nuclear',
    'Motores diésel':'oil_gas',
    'Turbina de gas':'oil_gas',
    'Turbina de vapor':'oil_gas',
    'Ciclo combinado':'oil_gas',
    'Otras renovables' : 'bioenergy',
    'Generación total' : 'others',
    'Cogeneración' : 'oil_gas',
    'Residuos no renovables': 'coal',
    'Residuos renovables':'bioenergy',
    'Turbinación bombeo':'hydropower',
    'Fuel + Gas' : 'oil_gas'
    }

In [73]:
gen['class'] = gen['class'].map(energy_dicc)

#### 4- units

According to the REE API, units for their values are MWh. In `spain_global`, not always are units given, and if they are, they are in MW. Therefore, to convert from MW to MWh, we simply need to multiply by the number of hours. As all requests to REE have been made for each month, we will multiply the capacity in `spain_global` by 720 (30 days * 24 hours) to unify units. After that, we will standardize all units to MWh in all dataframes for better understanding.

In [75]:
spain_global['capacity'] = spain_global['capacity'] * 720

In [76]:
global_web['capacity'] = global_web['capacity'] * 720

In [77]:
gen['unit'] = 'MWh'
demand['unit'] = 'MWh'
spain_global['unit'] = 'MWh'
global_web['unit'] = 'MWh'

#### 5- change columns names

Dataset "reservoir" has too long column names. We will replace them to shorter names.

In [78]:
dicc = {'DEMARCACIÓN HIDROGRÁFICA TERRESTRE':'zone',
       'Superficie total de los embalses de la Demaración Hidrográfica- km2' : 'surface_km_2'
        }

In [79]:
reservoir.columns = reservoir.columns.str.strip().map(lambda x: dicc.get(x, x))

In [80]:
reservoir.sample()

,zone,surface_km_2,aacc
14,EBRO,210.2,Aragon


## SAVING AS CSV FILES

In [81]:
gen.to_csv("data/gen.csv", index = False)
demand.to_csv('data/demand.csv', index = False)
spain_global.to_csv('data/spain_global.csv', index = False)
global_web.to_csv('data/global_web.csv', index = False)
reservoir.to_csv('data/reservoir.csv', index = False)